### Connect to Azure Machine Learning workspace

In [27]:
from azure.ai.ml import MLClient
from azure.identity import DefaultAzureCredential

credential = DefaultAzureCredential()
client = MLClient.from_config(credential)


Found the config file in: .\config.json


### Get Credentials from Key Vault

In [28]:
from azure.keyvault.secrets import SecretClient

keyvault_uri = "https://plato-default-key-vault.vault.azure.net/"
secret_client = SecretClient(vault_url=keyvault_uri, credential=credential)

tenant_id = secret_client.get_secret('azure-tenant-id').value
client_id = secret_client.get_secret('ml-client-id').value
client_secret = secret_client.get_secret('ml-client-secret').value

### Preprocess Tokenizer Trainer

In [29]:
from azure.ai.ml import command, Input, UserIdentityConfiguration
from azure.ai.ml import Input
from azure.ai.ml.constants import AssetTypes, InputOutputModes

identity = UserIdentityConfiguration()
registered_model_name = "plato-gpt-trained-model"

config_path = './config.json'
config_file_type = AssetTypes.URI_FILE
config_file_mode = InputOutputModes.RO_MOUNT

env = 'azureml:Plato-GPT-env:5'


# configure the command job
job = command(
    inputs=dict(
        config=Input(
            type=config_file_type,
            path=config_path,
            mode=config_file_mode
        ),
    ),
    environment_variables= {
        'AZURE_TENANT_ID': tenant_id,
        'AZURE_CLIENT_ID': client_id,
        'AZURE_CLIENT_SECRET': client_secret,
    },
    code="./src",  # location of source code
    command="python train.py --config ${{inputs.config}}",
    environment=env,
    compute='a100',
    instance_count=1,
    distribution={
        "type": "PyTorch",
        "process_count_per_instance": 4,
    },
    experiment_name="Plato-GPT-3",
    display_name="DDP-Train-50k-epoch",
    identity=identity
)

In [30]:
client.create_or_update(job)

Uploading src (0.07 MBs): 100%|##########| 65393/65393 [00:01<00:00, 58692.04it/s]




Experiment,Name,Type,Status,Details Page
Plato-GPT-3,gentle_ghost_bs3f3gqs7f,command,Starting,Link to Azure Machine Learning studio
